El objetivo de este script es extraer las imágenes RGB del WMS del Plan Nacional de Ortofotografía PNOA

El servicio Web Map (WMS) definido por el OGC (Open Geospatial Consortium) produce mapas de datos referenciados espacialmente, de forma dinámica a partir de información geográfica. Este estándar internacional define un "mapa" como una representación de la información geográfica en forma de un archivo de imagen digital. 
https://es.wikipedia.org/wiki/Web_Map_Service

Antes de comenzar con este proceso, hemos extraído de nuestra cartografía 40 shapefiles con extensión de 96x96 metros.
Los shapefile están compuesto por varios ficheros con el mismo nombre y distinta extensión.
Nosotros nos vamos a centrar en los .shp que contienen la información grafica.

Mas info: https://desktop.arcgis.com/es/arcmap/10.3/manage-data/shapefiles/shapefile-file-extensions.htm

La extensión de los shapefiles de 96x96 metros se obtiene de la resolución del pixel del PNOA (25cms/pixel) y las dimensiones de las imágenes (384,384). De cada shapefile se obtendrá una matriz de 10x10 imágenes. Y en total 4000 imágenes.

Para cada matriz correspondiente a un shapefile, vamos a localizar su coordenada inferior-izquierda. A partir de este punto iremos extrayendo las imágenes que se guardaran con un nombre que contiene la coordenada inferior-izquierda y la superior-derecha. De esta forma siempre mantendremos las coordenadas UTM de la imagen.

Extraeremos las imágenes con extensión 'jpg'


In [1]:
import shapefile
import os.path
from os import listdir
from owslib.wms import WebMapService

In [2]:
#URL del WMS del PNOA
URL = "https://www.ign.es/wms-inspire/PNOA-MA?SERVICE=WMS&REQUEST=GETCAPABILITIES"
#Acceso al servicio WMS mediante la URL del PNOA, version '1.1.1'
WMS = WebMapService(URL, version='1.1.1')
#Directorio de salida
OUTPUT_DIRECTORY = './imagenes/jpg/'

#Leemos ficheros directorio /shape
archivos=listdir("./shape")
for archivo in archivos:
    #Leemos solo ficheros extension .shp
    nombre, extension = os.path.splitext(archivo)
    if (extension == '.shp'):
        #Guardamos los datos del shapefile en variable sf
        sf = shapefile.Reader("./shape/"+archivo)   

        #Coordenadas UTM donde empieza la descarga, se extraen de las coordendas del shape.
        #bbox nos proporciona las coordendas de la caja delimitadora del shape
        x_min = round(sf.bbox[0], 2)
        y_min = round(sf.bbox[1], 2)

        #dx y df es la distancia en metros entre tiles
        #Debe ser el tamaño tile (size=384) por resolucion PNOA 0.25 cms/pixel, total 96 metros
        dx, dy = 96, 96

        #Total de numero de tiles del la matriz del mosaico x,y
        no_tiles_x = 10
        no_tiles_y = 10

        #Total imagenes mosaico
        total_no_tiles = no_tiles_x * no_tiles_y

        #Coordenadas maximas XY = valor minimo (x,y) + el numero de imagen*tamaño imagen
        x_max = x_min + no_tiles_x * dx
        y_max = y_min + no_tiles_y * dy

        #Cuadro imagenes
        BOUNDING_BOX = [x_min, y_min, x_max, y_max]

        #Extraccion WMS
        #Recorremos la matriz de las tiles en X
        for i in range(0, no_tiles_x):
            #Recorremos la matriz de la tiles en Y 
            for j in range(0, no_tiles_y):
                #calculamos coordenda x: x minima + numero tile por tamaño en metros
                ll_x_ = x_min + i*dx
                #calculamos coordenada y y x minima + numero tile por tamaño en metros
                ll_y_ = y_min + j*dy
                #coordenadas cuadro delimitador de la imagen, valores maximos es igual al minimo mas 96
                bbox = (ll_x_, ll_y_, ll_x_ + dx, ll_y_ + dy)
                #extraemos la imagenes del WMS
                #layers es la capa del WMS desde donde estraemos la imagen    
                img = WMS.getmap(layers = ['OI.OrthoimageCoverage'], 
                                 #srs es el sistema de coordenagas geografico. EPSG:25830 pertenece a ERTS89/30N provincia Alicante
                                 srs = 'EPSG:25830', 
                                 #bbox cuadro demilitador imagen a extraer
                                 bbox = bbox, 
                                 #size tamaño imagen (384*384)
                                 size = (384, 384), 
                                 #formato jpeg
                                 format = 'image/jpeg',
                                 #trasparent True 
                                 transparent = True)
                #nombre del fichero son las coordendas de cuadro delimitador, así siempre sabremos las coordendas de la imagen
                filename = "{}_{}_{}_{}.jpg".format(bbox[0], bbox[1], bbox[2], bbox[3])
                #salida de la imagen
                out = open(OUTPUT_DIRECTORY + filename, 'wb')
                #guardamos la imagen
                out.write(img.read())
                out.close()
        
        #Capa trabajada
        print("Capa finalizada " + archivo)

        #Fin del proceso
print("Proceso finalizado ")

Capa finalizada cons_10_960.shp
Capa finalizada cons_11_960.shp
Capa finalizada cons_12_960.shp
Capa finalizada cons_13_960.shp
Capa finalizada cons_14_960.shp
Capa finalizada cons_15_960.shp
Capa finalizada cons_16_960.shp
Capa finalizada cons_17_960.shp
Capa finalizada cons_18_960.shp
Capa finalizada cons_19_960.shp
Capa finalizada cons_1_960.shp
Capa finalizada cons_20_960.shp
Capa finalizada cons_21_960.shp
Capa finalizada cons_22_960.shp
Capa finalizada cons_23_960.shp
Capa finalizada cons_24_960.shp
Capa finalizada cons_25_960.shp
Capa finalizada cons_26_960.shp
Capa finalizada cons_27_960.shp
Capa finalizada cons_28_960.shp
Capa finalizada cons_29_960.shp
Capa finalizada cons_2_960.shp
Capa finalizada cons_30_960.shp
Capa finalizada cons_31_960.shp
Capa finalizada cons_32_960.shp
Capa finalizada cons_33_960.shp
Capa finalizada cons_34_960.shp
Capa finalizada cons_35_960.shp
Capa finalizada cons_36_960.shp
Capa finalizada cons_37_960.shp
Capa finalizada cons_38_960.shp
Capa final